## Récupération des informations générales du candidats

In [ ]:
# Librairie
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import numpy as np

# Configuration du navigateur Chrome
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Exécution sans interface graphique

ch_car = [str(i) for i in range(1, 201)]

list_egal1 = ["00" + i for i in ch_car if len(i) == 1]
list_egal2 = ["0" + i for i in ch_car if len(i) == 2]
list_egal3 = [i for i in ch_car if len(i) > 2]

liste_global = list_egal1 + list_egal2 + list_egal3



noms = []
stats = []
parti_candidat = []
statut = []
score = []
image = []
code_page = []

# Instance du navigateur Chrome
driver = webdriver.Chrome(options=options)

racine_url = "https://resultats.cei.ci/elections-municipales/circonscription/"

# Charger la page
for i in liste_global:
    driver.maximize_window()
    driver.get(racine_url + str(i))
    driver.implicitly_wait(10) 

    try:
        bouton = driver.find_element(By.XPATH, '//*[@id="radio-table"]')
        bouton.click()
        driver.implicitly_wait(10)

        for j in range(1, 100):
            candidat_xpath = f'/html/body/div/div/main/div/div/div[2]/div/div[2]/div[2]/div/div[3]/table/tbody/tr[{j}]/td[1]/div/div'
            parti_xpath = f'/html/body/div/div/main/div/div/div[2]/div/div[2]/div[2]/div/div[3]/table/tbody/tr[{j}]/td[2]/div/div'
            score_xpath = f'/html/body/div/div/main/div/div/div[2]/div/div[2]/div[2]/div/div[3]/table/tbody/tr[{j}]/td[3]'
            statut_xpath = f'/html/body/div/div/main/div/div/div[2]/div/div[2]/div[2]/div/div[3]/table/tbody/tr[{j}]/td[4]/div'
            img_xpath = f'/html/body/div/div/main/div/div/div[2]/div/div[2]/div[2]/div/div[3]/table/tbody/tr[{j}]/td[1]/div/img'
            driver.implicitly_wait(10)
            try:
                candidat = driver.find_element(By.XPATH, candidat_xpath)
                noms.append(candidat.text)
                
                parti = driver.find_element(By.XPATH, parti_xpath)
                parti_candidat.append(parti.text)
                
                nb_point = driver.find_element(By.XPATH, score_xpath)
                score.append(nb_point.text)
                
                gagnant = driver.find_element(By.XPATH, statut_xpath)
                statut.append(gagnant.text)
                
                img = driver.find_element(By.XPATH, img_xpath)
                img_src = img.get_attribute("src")
                image.append(img_src)
                
                code_page.append(i)
                
            except NoSuchElementException:
                break

    except NoSuchElementException:
        pass  

# Fermer le navigateur
driver.quit()
df = pd.DataFrame({"noms":noms, "parti":parti_candidat, "score":score, "statut":statut, "photo":image, "code":code_page})
df.head()

## Récupération des statistiques de la campagne

In [ ]:
# Configuration du navigateur Chrome
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Exécution sans interface graphique

def supp_espace(l):
    nv_liste = []
    for i in range(0, len(l)):
        nv_liste.append(l[i].replace("\u202f", ""))
    return nv_liste

inscrit = []
total = []
taux = []
bul_nul = []
bul_blanc = []
suff_exp = []
index_page = []

# Instance du navigateur Chrome
driver = webdriver.Chrome(options=options)

racine_url = "https://resultats.cei.ci/elections-municipales/circonscription/"

# Charger la page
for i in liste_global:
    driver.maximize_window()
    driver.get(racine_url + str(i))
    driver.implicitly_wait(10)  # Attendre que la page charge complètement
    bouton = driver.find_element(By.XPATH, '//*[@id="radio-table"]')
    bouton.click()
    
    inscrit_xpath = driver.find_element(By.XPATH, '/html/body/div/div/main/div/div/div[2]/div/div[2]/div[2]/div/div[1]/div[1]/p[1]')
    total_xpath = driver.find_element(By.XPATH, '/html/body/div/div/main/div/div/div[2]/div/div[2]/div[2]/div/div[1]/div[2]/p[1]')
    tx_xpath = driver.find_element(By.XPATH, '/html/body/div/div/main/div/div/div[2]/div/div[2]/div[2]/div/div[1]/div[3]/p[1]')
    bul_nul_xpath = driver.find_element(By.XPATH, '/html/body/div/div/main/div/div/div[2]/div/div[2]/div[2]/div/div[1]/div[4]/p[1]')
    bul_blanc_xpath = driver.find_element(By.XPATH, '/html/body/div/div/main/div/div/div[2]/div/div[2]/div[2]/div/div[1]/div[5]/p[1]')
    suffrage_exprime_xpath = driver.find_element(By.XPATH, '/html/body/div/div/main/div/div/div[2]/div/div[2]/div[2]/div/div[1]/div[6]/p[1]')
    
    driver.implicitly_wait(10)

    inscrit.append(inscrit_xpath.text)
    total.append(total_xpath.text)
    taux.append(tx_xpath.text)
    bul_nul.append(bul_nul_xpath.text)
    bul_blanc.append(bul_blanc_xpath.text)
    suff_exp.append(suffrage_exprime_xpath.text)
    
    index_page.append(i)

# Fermer le navigateur
driver.quit()


inscrit = supp_espace(inscrit)
total = supp_espace(total)
taux = supp_espace(tx)
bulletin_nul = supp_espace(bul_nul)
bulletin_blanc = supp_espace(bul_blanc)
suffrage_exprime = supp_espace(suff_exp)